<a href="https://colab.research.google.com/github/pinyu1122/08135024githw1/blob/main/%E7%88%AC%E5%8F%96google_map_%E6%98%9F%E6%95%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

執行程式碼前，記得先上傳 實踐美食.csv

In [1]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,692 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,798 kB]
Get:13 https://developer.downloa

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True
wd = webdriver.Chrome(options=chrome_options)

In [9]:
# 匯入必要函式
import time  #用來讓程式暫停 (sleep) 一段時間，確保網頁內容載入完成後再抓取資料。
import pandas as pd #用來讀取與儲存 CSV 檔案，方便處理餐廳名稱與評分資料。
import requests     #通常用來發送 HTTP 請求（這裡沒用到，但可以作為爬蟲備用）。
from bs4 import BeautifulSoup #用來解析 HTML，雖然目前程式主要用 Selenium，但 BeautifulSoup 仍然是網頁爬蟲的常見工具。
from selenium import webdriver    #主要用來模擬瀏覽器行為，自動打開 Google Maps 並抓取評分。
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import chromedriver_autoinstaller   #自動安裝並設定 ChromeDriver，確保 Selenium 能正常運作。

# 安裝並啟動 ChromeDriver
chromedriver_autoinstaller.install()  #自動安裝適合目前環境的 ChromeDriver，確保 Selenium 可以驅動 Chrome 瀏覽器。
options = webdriver.ChromeOptions() #設定瀏覽器選項
options.add_argument('--headless')  # 無頭模式，不開啟瀏覽器視窗，適合 Colab 環境。
options.add_argument('--no-sandbox')  # Selenium 以無沙盒模式執行，避免權限問題（Colab 需要）。
options.add_argument('--disable-dev-shm-usage') #防止 Chrome 使用 /dev/shm，避免記憶體不足導致崩潰。

# 啟動 WebDriver
browser = webdriver.Chrome(options=options)  #啟動 Chrome 瀏覽器（背景執行）。

# 讀取 CSV 檔案
dfc = pd.read_csv('實踐 美食.csv', encoding='utf-8')

# 取得餐廳名稱
name_crawler = dfc['name'].tolist() #將 name 欄位的所有餐廳名稱存成 Python list，方便後續爬取。

# 存放評分結果
stars = []

# 爬取 Google Maps 評分
for nc in name_crawler:
    url = f"https://www.google.com.tw/maps/search/{nc}"  # Google Maps 搜尋結果

    browser.get(url)
    time.sleep(3)  # 等待 3 秒，確保網頁加載完成，避免 Selenium 在頁面還沒完全載入時就嘗試抓取資料（可能會報錯）

#try:：防止 Selenium 因為找不到元素而直接報錯，確保程式能夠繼續執行。
    try:
        # 使用 XPath 直接尋找評分（最新結構）
        star_element = browser.find_element(By.XPATH, '//span[@class="MW4etd"]')#這個 XPath (//span[@class="MW4etd"]) 來自 Google Maps 的 HTML 結構。
                                              #find_element() 會找到 第一個 符合條件的標籤

        if star_element:
            star = star_element.text # star_element.text：取得評分數字（如 4.6）。
        else:
            star = 'N/A'  # 若未找到評分，則標記為 N/A

#處理錯誤(如果 browser.find_element() 失敗（例如找不到評分），程式不會崩潰而是：
# star = 'Error'：將該餐廳的評分標記為 "Error"。
# print(f"無法抓取 {nc} 的評分，錯誤訊息：{e}")：顯示錯誤訊息。)
    except Exception as e:
        star = 'Error'
        print(f"無法抓取 {nc} 的評分，錯誤訊息：{e}")


    stars.append(star)
    print(f"{nc} 的評分: {star}") #即時顯示每間餐廳的評分。

# 關閉瀏覽器
browser.quit()

# 儲存結果
dfc['Stars'] = stars
dfc.to_csv('美食評分結果.csv', encoding='utf-8-sig', index=False) #index=False 避免輸出時增加不必要的索引欄位。
print("已儲存結果至 '美食評分結果.csv'")

林文必涼麵 的評分: 4.4
兄弟麵館 的評分: 4.0
麻古茶坊大直店 的評分: 2.6
PaPaPasta帕帕麵店 的評分: 4.1
佳佳牛排館 的評分: 4.0
勝拉麵 的評分: 4.2
大直創義麵 的評分: 4.1
揚屋大眾日式料理 的評分: 4.8
無法抓取 小森林餐坊 的評分，錯誤訊息：Message: no such element: Unable to locate element: {"method":"xpath","selector":"//span[@class="MW4etd"]"}
  (Session info: chrome=134.0.6998.88); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5954ef1c0a1a <unknown>
#1 0x5954eec78390 <unknown>
#2 0x5954eecc9c85 <unknown>
#3 0x5954eecc9eb1 <unknown>
#4 0x5954eed18d64 <unknown>
#5 0x5954eecefbfd <unknown>
#6 0x5954eed1607b <unknown>
#7 0x5954eecef9a3 <unknown>
#8 0x5954eecbb60e <unknown>
#9 0x5954eecbcdd1 <unknown>
#10 0x5954ef186ddb <unknown>
#11 0x5954ef18acbc <unknown>
#12 0x5954ef16e392 <unknown>
#13 0x5954ef18b834 <unknown>
#14 0x5954ef1521ef <unknown>
#15 0x5954ef1af038 <unknown>
#16 0x5954ef1af216 <unknown>
#17 0x5954ef1bf896 <unknown>
#18 0x7e465f452ac3 <unknown>

小森林